#Тестовое задание в Bewise.ai
Горожанкин Егор

Для данной работы было использована библиотека Spacy, так как она показала лучший результат для обработки spech recognition text

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Так как в данном тексте не учитывается регистр слов, то потребовалось дополнительно капитализировать имена собственные.

In [2]:
!git clone https://github.com/Raven-SL/ru-pnames-list.git

fatal: destination path 'ru-pnames-list' already exists and is not an empty directory.


In [3]:
import spacy
import csv
import pandas


Импорт тестового файла осуществляется при помощи google drive. Для подгрузки другого файла измените путь к нему.

In [4]:
from google.colab import drive
drive.mount('/content/drive')
df = pandas.read_csv('/content/drive/MyDrive/test_data.csv')
nlp = spacy.load("ru_core_news_lg")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


Функция для получения словаря имен, где ключом является первая буква. Это должно ускорить поиск слов в словаре.

In [5]:
def get_name_dict():
    names_list = {}
    female_file = open('/content/ru-pnames-list/lists/female_names_rus.txt')
    for i in female_file:
        if len(i) == 1:
            break
        now = i.lower()[:-1]
        if names_list.get(now[0]) is None:
            names_list[now[0]] = [now]
        else:
            names_list[now[0]].append(now)

    male_file = open('/content/ru-pnames-list/lists/male_names_rus.txt')
    for i in male_file:
        if len(i) == 1:
            break
        now = i.lower()[:-1]
        if names_list.get(now[0]) is None:
            names_list[now[0]] = [now]
        else:
            names_list[now[0]].append(now)
    return names_list

Так как для данного задания не особо требуются реплики клиента, то оставлены лишь реплики менеджера.

In [6]:
# получение текста менеджера из диалога n
def get_manager_text_in_dialog_n(df, n):
    names_dict = get_name_dict()
    temp_data = df[df['dlg_id'] == n][df['role'] == 'manager']['text'].values
    res = []
    for sentence in temp_data:
        for word in sentence.split():
            if word[0] in names_dict.keys() and word in names_dict[word[0]]:
                sentence = sentence.replace(word, word.capitalize())
        res.append(sentence)
    return res

Для поиска названия компания стандартные методы извлечения сущностей не работали (а датасета для обучения модели нет), то были выбраны слова, определяющие разговор о самой компании (из логики, что если было произнесено слово "компания", то должно и прозвучать название). Далее добавляется первое слово, а потом идет добавление, пока не встретится глагол или наречие.

In [7]:
company_word_list = 'компания организация ооо стартап фирма ип'.split()
def find_company_name(text):
    doc = nlp(text)
    check = False
    res = ''
    for token in doc:
        if token.lemma_ in company_word_list:
            check = True
            continue
        if check:
            if res == '':
                res = token.text
            elif token.pos_ != 'VERB' and token.pos_ != 'ADV': # глагол
                res += ' ' + token.text
            else:
                return res
    if res == '':
        return None
    return res

Извлечение имен (после их капитализации) работает хорошо, но требуется определить, идет ли разговор о клиенте или это представляется менеджер. Для этого определяем, в каком лице говорятся соседние слова, и если встречается первое, то это и есть менеджер. В противном случае продолжаем просматривать текст.

In [8]:
def find_manager_name(doc):
    ents_list = []
    for token in doc.ents:
        ents_list.append([token.start_char, token.text])
    now_in_ents_list = 0
    check = False
    for token in doc:
        if 'Person=First' in token.morph:
            if now_in_ents_list < len(ents_list):
                return ents_list[now_in_ents_list][1]
            return None
        elif 'Person=Second' in token.morph or 'Person=Third' in token.morph:
            check = True
            now_in_ents_list += 1

    if check is False and len(ents_list) > 0:
        return ents_list[now_in_ents_list][1]
    return None

Для определения приветствия и прощания, аналогично как и с компанией, был взят стэминг самых популярных слов, с которые и будут проверять вхождение в слово. 

In [9]:
def find_greeting(doc):
    greetings_words = 'привет добрый здрав'.split()

    for token in doc:
        for greetings in greetings_words:
            if greetings in token.lemma_:
                return True
    return None



In [10]:
def find_farewell(doc):
    farawell_words = 'пока свидан всего'.split()

    for token in doc:
        for farawell in farawell_words:
            if farawell in token.lemma_:
                return True
    return None

В самом вызове мы проверяем лишь первые и последние 5 реплик, так как в остальном тексте встреча проверяемых сущностей будет нелогично и являться ошибкой оператора.

Требование проверяется по логике, что менеджер поздаровался и попрощался (ему не обязательно представляться), так как не совсем точно понятно из тз.

Результатом анализа является словарь для каждого диалога со всеми проверяемыми полями.

In [11]:
data = []
data_keys = '''имя менеджера,название компании,приветствие,прощание,реплика с имененем менеджера,требование'''.split(',')
lemmatizer = nlp.get_pipe("lemmatizer")
    
for i in range(0, 6):
    temp_dialog = get_manager_text_in_dialog_n(df, i)
    temp_dict = {}
    for key in data_keys:
        temp_dict[key] = ''
    
    for token in temp_dialog[:5] + temp_dialog[-5:]:
        doc = nlp(token)
        temp_res = [find_manager_name(doc), find_company_name(doc), find_greeting(doc), find_farewell(doc)]

        for iter in range(len(temp_res)):
            if temp_dict.get(data_keys[iter]) == '' and temp_res[iter] is not None:
                temp_dict[data_keys[iter]] = temp_res[iter]
                if iter == 0:
                    temp_dict[data_keys[-2]] = token
    temp_dict['требование'] = temp_dict['приветствие'] == True and temp_dict['прощание'] == True
    data.append(temp_dict)
for i in range(len(data)):
    print('Диалог ' + str(i))
    for key, value in data[i].items():
        print(key, ': ', value, sep='')
    print()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Диалог 0
имя менеджера: Ангелина
название компании: диджитал бизнес
приветствие: True
прощание: True
реплика с имененем менеджера: Меня зовут Ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
требование: True

Диалог 1
имя менеджера: Ангелина
название компании: диджитал бизнес
приветствие: True
прощание: True
реплика с имененем менеджера: Меня зовут Ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
требование: True

Диалог 2
имя менеджера: Ангелина
название компании: диджитал бизнес
приветствие: True
прощание: 
реплика с имененем менеджера: Меня зовут Ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
требование: False

Диалог 3
имя менеджера: Алло Дмитрий
название компании: китобизнес
приветствие: True
прощание: True
реплика с имененем менеджера: Алло Дмитрий добрый день
требование